# Section 3 - Alembic for Database Management 
## Initializing Connection to Database

Firstly, lets once again initialize the database connection (copy from the previous section)

In [ ]:
from sqlalchemy import create_engine, URL
from sqlalchemy.orm import sessionmaker

url = URL.create(
    drivername="postgresql+psycopg2",  # driver name = postgresql + the library we are using (psycopg2)
    username='testuser',
    password='testpassword',
    host='localhost',
    database='testuser',
    port=5432
)

engine = create_engine(url, echo=True)
session_pool = sessionmaker(bind=engine)

If you for some reason skipped the previous section, you can run the following code to declare the tables.

In [ ]:
from typing_extensions import Annotated
from typing import Optional
from sqlalchemy.ext.declarative import declared_attr

import datetime
from sqlalchemy.dialects.postgresql import TIMESTAMP
from sqlalchemy import ForeignKey, BIGINT

from sqlalchemy import String
from sqlalchemy.orm import Mapped
from sqlalchemy.orm import mapped_column
from sqlalchemy.sql.functions import func

from sqlalchemy import INTEGER
from sqlalchemy.orm import DeclarativeBase


# Creating a base class
class Base(DeclarativeBase):
    pass

# Users ForeignKey
user_fk = Annotated[
    int, mapped_column(BIGINT, ForeignKey("users.telegram_id", ondelete="CASCADE"))
]

# integer primary key
int_pk = Annotated[int, mapped_column(INTEGER, primary_key=True)]

# string column with length 255
str_255 = Annotated[str, mapped_column(String(255))]


class TimestampMixin:
    created_at: Mapped[datetime] = mapped_column(TIMESTAMP, server_default=func.now())
    updated_at: Mapped[datetime] = mapped_column(TIMESTAMP, server_default=func.now(), onupdate=func.now())


class TableNameMixin:
    @declared_attr.directive
    def __tablename__(cls) -> str:
        return cls.__name__.lower() + "s"
    
# So now your Users class will look like this:
class Users(Base, TimestampMixin, TableNameMixin):

    telegram_id: Mapped[int] = mapped_column(BIGINT, primary_key=True, autoincrement=False)
    full_name: Mapped[str_255]
    username: Mapped[Optional[str_255]]
    language_code: Mapped[str_255]
    referrer_id: Mapped[Optional[user_fk]]




class Orders(Base, TimestampMixin, TableNameMixin):

    order_id: Mapped[int_pk]
    user_id: Mapped[user_fk]


class Products(Base, TimestampMixin, TableNameMixin):
    product_id: Mapped[int_pk]
    title: Mapped[str_255]
    description: Mapped[str]


class OrderProducts(Base, TableNameMixin):

    order_id: Mapped[int] = mapped_column(INTEGER, ForeignKey("orders.order_id", ondelete="CASCADE"), primary_key=True)
    product_id: Mapped[int] = mapped_column(INTEGER, ForeignKey("products.product_id", ondelete="RESTRICT"), primary_key=True)
    quantity: Mapped[int]

### Creating a database

Now you can use SQLAlchemy to create a database.

**Although IT IS NOT RECOMMENDED**, since you would like to track changes in the database, and tracking changes with
SQLAlchemy boils down to writing them as raw SQL statements, which is not convenient, i would like to show you how to
create your tables with just SQLAlchemy.

We will use Alembic to create tables a little later.

In [ ]:
# You can drop all tables by running the following code:
Base.metadata.drop_all(engine)

# And to create all tables:
Base.metadata.create_all(engine)

Congratulations! You have created your first database with SQLAlchemy!

But this is actually not the *correct* way to do it.

We are going to do it with alembic, which is a migration tool for SQLAlchemy.

## Using Alembic to create migrations.

As we discussed earlier, we will need to use [Alembic](https://alembic.sqlalchemy.org/en/latest/tutorial.html) to create
tables and track changes in the database.

> **Note:**
> This history of changes is called **migrations**.

In order to use Alembic, you need to install it. Make sure you are using the virtual environment, or use this Jupyter Notebook cmd to install it:

In [ ]:
!pip install alembic
!alembic init alembic

Now there were generated many files in the `alembic` directory. Please
read [their tutorial](https://alembic.sqlalchemy.org/en/latest/tutorial.html) to understand how they work.

In order for the alembic to connect to our database we need to add the following to the `env.py` file.

```python
url = URL.create(
    drivername="postgresql+psycopg2",  # driver name = postgresql + the library we are using
    username='testuser',
    password='testpassword',
    host='localhost',
    database='testuser',
    port=5432
)
config.set_main_option('sqlalchemy.url', str(url))
```

By this code we are replacing the `sqlalchemy.url` from the `alembic.ini` file with the `url` we created earlier.

### Environment variables
We actually don't want to store our database connection information in the code, so we are going to use environment
variables to store them.

So, you would need to create a `.env` file in the root directory of the project, and add the following lines to it:

```dotenv
POSTGRES_USER=testuser
POSTGRES_PASSWORD=testpassword
POSTGRES_DB=testuser
DATABASE_HOST=localhost
```

Now, we can use a library called `environs` to get the environment variables from the `.env` file.

```python
from environs import Env

env = Env()
env.read_env()
```

So you can replace the `url` variable with the following code:

```python
url = URL.create(
    drivername="postgresql+psycopg2",  # driver name = postgresql + the library we are using
    username=env.str('POSTGRES_USER'),
    password=env.str('POSTGRES_PASSWORD'),
    host=env.str('DATABASE_HOST'),
    database=env.str('POSTGRES_DB'),
    port=5432
)

config.set_main_option('sqlalchemy.url', str(url))
```

Again, what we are actually doing here, is replacing the `sqlalchemy.url` from the `alembic.ini` file with the `url` we created
earlier.

You can also write your `sqlalchemy.url` string in the `alembic.ini` file if you want (not recommended).

> **Note:**
>
> You can also store your database connection information (login, password, port, ...) in environment variables and
> get them from there in the `env.py` file.

Now, for alembic to see our tables, we need to assign `target_metadata` to the metadata from the `Base` class, imported
from our main file to the `env.py` file.

```python
from main import Base

# target_metadata = None REPLACE WITH:
target_metadata = Base.metadata
```

However, what we are going to do, is to generate a first migration from our code, it will generate more python code in
the `alembic/versions` directory.

In [ ]:
# but first, we need to delete the existing tables with SQLAlchemy
Base.metadata.drop_all(engine)

Now we are ready to generate our first automatic migration.

Alembic has a command line interface, which we can use to generate migrations. Here are some examples:

### Create a Migration
To create a new migration script, use the `revision` command:
```bash
alembic revision -m "your message"
```
Replace `"your message"` with a brief description of the changes the migration will make. This will create a new file in the `alembic/versions` directory.

### Editing a Migration
Open the newly created file and locate the `upgrade()` and `downgrade()` functions. `upgrade()` is applied when the migration runs, and `downgrade()` is applied when a migration is undone.

For example, to add a new table:

`your_message_1fndmq03f.py` - the name of the migration file in the `alembic/versions` directory
```python
def upgrade():
    op.create_table('my_new_table',
                    Column('id', Integer, primary_key=True),
                    Column('name', String))

def downgrade():
    op.drop_table('my_new_table')
```

### Run Migrations
To apply all pending migrations, run:
```bash
alembic upgrade head
```

### Undo Migrations
To undo the last migration, run:
```bash
alembic downgrade -1
```
To undo all migrations and return to the start, run:
```bash
alembic downgrade base
```

### Show Current Migration Status
To view the current migration status, run:
```bash
alembic current
```

### Show Migration History
To view all the migrations and their status (upgraded or not), run:
```bash
alembic history
```

This is a very basic introduction to Alembic's CLI. For more detailed usage, please refer to the [official Alembic documentation](https://alembic.sqlalchemy.org/en/latest/).

### Automatic Migrations
Alembic supports autogeneration of migration scripts from the current state of your SQLAlchemy models. This is accomplished using the `--autogenerate` option on the `revision` command. Here's how you can use it:

### Generate a Migration Script Automatically
Before creating an automatic migration script, ensure that your SQLAlchemy models' current state matches the database's state. Alembic will compare the current state of your models against the current state of your database and generate a migration script to reconcile the differences.

To generate an automatic migration script, use the following command:

```bash
alembic revision --autogenerate -m "description of changes"
```
Replace "description of changes" with a brief description of the changes made to your models.

For example, if you added a `User` model to your SQLAlchemy models, your command would look like this:

```bash
alembic revision --autogenerate -m "added user model"
```

This will create a new migration script file under the `alembic/versions` directory.

### Check the Generated Script
Alembic's autogenerate feature isn't perfect and won't catch every change you make to your models. Therefore, it's crucial to check the generated script and make sure it matches the changes you've made to your models.

Open the new script file and verify the `upgrade()` and `downgrade()` functions reflect the changes you've made to your models.

So let's generate our first automatic migration:

In [ ]:
!alembic revision --autogenerate -m "initial migration"

Cool! Right now our migration looks like this:

```python
"""initial migration

Revision ID: dcca43073c21
Revises:
Create Date: 2023-05-08 18:15:04.835447

"""
from alembic import op
import sqlalchemy as sa
from sqlalchemy.dialects import postgresql

# revision identifiers, used by Alembic.
revision = 'dcca43073c21'
down_revision = None
branch_labels = None
depends_on = None


def upgrade() -> None:
    # ### commands auto generated by Alembic - please adjust! ###
    op.create_table('products',
    sa.Column('product_id', sa.INTEGER(), nullable=False),
    sa.Column('title', sa.String(length=255), nullable=False),
    sa.Column('description', sa.String(), nullable=False),
    sa.Column('created_at', postgresql.TIMESTAMP(), server_default=sa.text('now()'), nullable=False),
    sa.Column('updated_at', postgresql.TIMESTAMP(), server_default=sa.text('now()'), nullable=False),
    sa.PrimaryKeyConstraint('product_id')
    )
    op.create_table('users',
    sa.Column('telegram_id', sa.BIGINT(), nullable=False),
    sa.Column('full_name', sa.String(length=255), nullable=False),
    sa.Column('username', sa.String(length=255), nullable=True),
    sa.Column('language_code', sa.String(length=255), nullable=False),
    sa.Column('referrer_id', sa.BIGINT(), nullable=False),
    sa.Column('created_at', postgresql.TIMESTAMP(), server_default=sa.text('now()'), nullable=False),
    sa.Column('updated_at', postgresql.TIMESTAMP(), server_default=sa.text('now()'), nullable=False),
    sa.ForeignKeyConstraint(['referrer_id'], ['users.telegram_id'], ondelete='CASCADE'),
    sa.ForeignKeyConstraint(['referrer_id'], ['users.telegram_id'], ondelete='CASCADE'),
    sa.PrimaryKeyConstraint('telegram_id')
    )
    op.create_table('orders',
    sa.Column('order_id', sa.INTEGER(), nullable=False),
    sa.Column('user_id', sa.BIGINT(), nullable=False),
    sa.Column('created_at', postgresql.TIMESTAMP(), server_default=sa.text('now()'), nullable=False),
    sa.Column('updated_at', postgresql.TIMESTAMP(), server_default=sa.text('now()'), nullable=False),
    sa.ForeignKeyConstraint(['user_id'], ['users.telegram_id'], ondelete='CASCADE'),
    sa.ForeignKeyConstraint(['user_id'], ['users.telegram_id'], ondelete='CASCADE'),
    sa.PrimaryKeyConstraint('order_id')
    )
    op.create_table('order_products',
    sa.Column('order_id', sa.INTEGER(), nullable=False),
    sa.Column('product_id', sa.INTEGER(), nullable=False),
    sa.Column('quantity', sa.Integer(), nullable=False),
    sa.ForeignKeyConstraint(['order_id'], ['orders.order_id'], ondelete='CASCADE'),
    sa.ForeignKeyConstraint(['product_id'], ['products.product_id'], ondelete='RESTRICT'),
    sa.PrimaryKeyConstraint('order_id', 'product_id')
    )
    # ### end Alembic commands ###


def downgrade() -> None:
    # ### commands auto generated by Alembic - please adjust! ###
    op.drop_table('order_products')
    op.drop_table('orders')
    op.drop_table('users')
    op.drop_table('products')
    # ### end Alembic commands ###
```

You can see here 2 functions:

- upgrade - which will create the tables, and add the columns, constraints, etc.
- downgrade - which will drop the tables, and remove the columns, constraints, etc., that are applied in the `upgrade`
  function.

So, whenever you want to upgrade your database, you can just run the `alembic upgrade head` command.

### Updating and downgrading

You can also update and downgrade your database by relative to current migration scripts (`+N` or `-N` migrations):

```bash
# upgrade
alembic upgrade +1
# or downgrade
alembic downgrade -1
```

Let's apply our changes to the database.

In [ ]:
!alembic upgrade head


It got none rows because we haven't created any user in our database. However, it works!

### Changes in the database

Whenever you like to change your tables, you can do it by editing your python code of table classes, or otherwise you
can create a new migration script with `alembic revision --autogenerate -m "new migration"` and add the changes
manually.

This is because not always the alembic will understand what you are trying to do. Let's compare two examples.

#### Changing columns data type

- What if we need to change language_code data type to VARCHAR with a new length? 255 is more than enough, so let's
  change to 10 characters.
- What if we need to restrict the length of the product description from TEXT with unfixed max length to fixed max
  length of 3000 characters?

We will edit that in our classes:

```python
from sqlalchemy import DECIMAL

class User(Base):
    ...
    # username: Mapped[Optional[str_255]]
    # language_code: Mapped[str] = mapped_column(VARCHAR(255)) 

    # We replace with
    user_name: Mapped[Optional[str_255]]]
    language_code: Mapped[str] = mapped_column(VARCHAR(10))


class Product(Base):
    # description: Mapped[Optional[str]]
    description: Mapped[Optional[str]] = mapped_column(VARCHAR(3000))
    price: Mapping[float] = mapped_column(DECIMAL(precision=16, scale=4))

```

Now let's create a new migration script:

```bash
alembic revision --autogenerate -m "changed products and users"
```

You will see that no changes were found by alembic:
```python
def upgrade() -> None:
    # ### commands auto generated by Alembic - please adjust! ###
    pass
    # ### end Alembic commands ###


def downgrade() -> None:
    # ### commands auto generated by Alembic - please adjust! ###
    pass
    # ### end Alembic commands ###
```

So in that case we need to edit the code manually:

```python
def upgrade() -> None:
    # ### commands auto generated by Alembic - please adjust! ###
    op.alter_column('users', 'language_code', type_=sa.VARCHAR(10), nullable=False)
    op.alter_column('users', 'username', new_column_name='user_name')

    op.alter_column('products', 'description', type_=sa.VARCHAR(3000))
    op.add_column('products', sa.Columnt('price', sa.DECIMAL(precision=16, scale=4), nullable=False)
    # ### end Alembic commands ###


def downgrade() -> None:
    # ### commands auto generated by Alembic - please adjust! ###
    op.alter_column('products', 'description', type_=sa.VARCHAR(), nullable=True)
    op.drop_column('products', 'price')
    op.alter_column('users', 'language_code', type_=sa.VARCHAR(255), nullable=False)
    op.alter_column('users', 'user_name', new_column_name='username')
    # ### end Alembic commands ###
```

Now we have our newly created migration script, and we need to upgrade our database.

```bash
alembic upgrade head
```

OR you can upgrade just one revision:

```bash
alembic upgrade +1
```

#### Adding a new column

What if we need to add a new column to our table. Let's see another example.

Let's add a new column to our table:

```python
class Users(Base):
    ...
    phone_number: Mapped[Optional[str]] = mapped_column(VARCHAR(50))
```

Let's create a new migration script:

```bash
alembic revision --autogenerate -m "added user phone number"
```

Hurray! This change was detected by alembic, and we can upgrade our database.

```python
def upgrade() -> None:
    # ### commands auto generated by Alembic - please adjust! ###
    op.add_column('users', sa.Column('phone_number', sa.VARCHAR(length=50), nullable=True))
    # ### end Alembic commands ###


def downgrade() -> None:
    # ### commands auto generated by Alembic - please adjust! ###
    op.drop_column('users', 'phone_number')
    # ### end Alembic commands ###
```

Let's apply our changes to the database.

```bash
alembic upgrade +1
```

Brilliant! You have learnt how to create tables with Alembic and SQLAlchemy, and how to upgrade and downgrade your
database.